In [1]:
from datetime import date, timedelta

from featureeng.times import get_trading_times_for_london_ny
from featureeng.features import *
import featureeng as feng
import data.readers as rd
import joblib


import matplotlib.pyplot as plt

import multiprocessing
import time
import threading
import json
import os

import pyarrow as pa
import pyarrow.parquet as pq

# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")

# Classifier/Regressor
from xgboost import XGBClassifier

# Feature selection
from BorutaShap import BorutaShap

# Data processing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.mixture import GaussianMixture

import pickle
# Validation
from sklearn.model_selection import StratifiedKFold

root_dir = '/home/dcai/data/features'
instrument='EURCHF'
feature_set="4"



temp_save_dir="{}/{}".format(root_dir,instrument)

def save_df(save_dir, df, name):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    pq.write_table(pa.Table.from_pandas(df), os.path.join(save_dir, name))


temp_save_dir="{}/{}".format(root_dir,instrument)
def read_parquet(save_dir, name):
    return pq.ParquetFile(os.path.join(save_dir, name)).read().to_pandas()


with open(os.path.join(temp_save_dir, 'y_stratified.pickle'), 'rb') as f:
    y_stratified = pickle.load(f)

    
def read_pickle(path, file_name):
    with open(os.path.join(path, file_name), 'rb') as f:
        return pickle.load(f)


#allow logloss and classification error plots for each iteraetion of xgb model
def plot_compare(metrics,eval_results,epochs):
    for m in metrics:
        test_score = eval_results['val'][m]
        train_score = eval_results['train'][m]
        rang = range(0, epochs)
        plt.rcParams["figure.figsize"] = [6,6]
        plt.plot(rang, test_score,"c", label="Val")
        plt.plot(rang, train_score,"orange", label="Train")
        title_name = m + " plot"
        plt.title(title_name)
        plt.xlabel('Iterations')
        plt.ylabel(m)
        lgd = plt.legend()
        plt.show()    

In [2]:
# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")

# Classifier/Regressor
from xgboost import XGBRegressor, XGBClassifier, DMatrix

# Model selection

from sklearn.model_selection import KFold, StratifiedKFold

# Metrics

from sklearn.metrics import mean_squared_error, auc, f1_score, roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer

# Data processing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



In [4]:
### Load the data
X_train = read_parquet(temp_save_dir, "trainX.parqet")
y_train = pd.DataFrame(read_pickle(temp_save_dir, "trainY.pickle"), columns=['target']).target

X_valid = read_parquet(temp_save_dir, "validateX.parqet")
y_valid = pd.DataFrame(read_pickle(temp_save_dir, "validateY.pickle"), columns=['target']).target

eval_set = [(X_valid, y_valid)]


In [5]:
X_train.columns

Index(['MeanPrice', 'close_AVG_1D_20', 'close_AVG_1D_5', 'close_AVG_1D_60',
       'close_AVG_1H_20', 'close_AVG_1H_5', 'close_AVG_1H_60',
       'close_MAX_1D_20', 'close_MAX_1D_5', 'close_MAX_1D_60',
       ...
       'rolling_1D_60_3_2', 'rolling_1D_60_3_3', 'rolling_4H_20_4_1',
       'rolling_4H_20_4_2', 'rolling_4H_20_4_3', 'rolling_4H_20_4_4',
       'rolling_4H_60_20_4_1', 'rolling_4H_60_20_4_2', 'rolling_4H_60_20_4_3',
       'rolling_4H_60_20_4_4'],
      dtype='object', length=134)

In [6]:

# Stratifying the target
y_stratified = pd.cut(y_train.rank(method='first'), bins=10, labels=False)



In [7]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title="model", callbacks=None, eval_set=None, eval_metric=['auc'], early_stopping_rounds=50):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    
    if callbacks is not None:
        optimizer.fit(X, y, callback=callbacks, eval_set=eval_set, eval_metric=eval_metric, verbose=20)
    else:
        optimizer.fit(X, y, eval_set=eval_set, early_stopping_rounds=early_stopping_rounds, verbose=20)
        
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           + u"\u00B1"+" %.3f") % (time() - start, 
                                   len(optimizer.cv_results_['params']),
                                   best_score,
                                   best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [8]:
scoring = make_scorer(f1_score, average='weighted', 
                      greater_is_better=True)



In [9]:
# Setting the validation strategy
skf = StratifiedKFold(n_splits=3,
                      shuffle=False)

cv_strategy = list(skf.split(X_train, y_stratified))

In [10]:
# Setting the basic regressor
reg = XGBClassifier(random_state=0, booster='gbtree', objective='multi:softprob', tree_method='gpu_hist')

In [11]:
# Setting the search space
search_spaces = {'learning_rate': [0.01, 0.025, 0.05, 0.08],
                 'max_depth': Integer(3, 10),
                 'verbose_eval' : [10],
                 'subsample': [0.5, 0.7, 0.75, 0.8, 0.85, 0.9],
                 'colsample_bytree':[0.5, 0.7, 0.75, 0.8, 0.85, 0.9], # subsample ratio of columns by tree
                 'reg_lambda': Real(1e-9, 30., 'uniform'), # L2 regularization
                 'reg_alpha': Real(1e-9, 0.5, 'uniform'), # L1 regularization
                 'n_estimators': Integer(50, 900)
   }

In [12]:
# Wrapping everything up into the Bayesian optimizer
opt = BayesSearchCV(estimator=reg,
                    verbose=20,
                    search_spaces=search_spaces,                      
                    scoring=scoring,                                  
                    cv=cv_strategy,                                           
                    n_iter=300,                                       # max number of trials
                    n_points=1,                                       # number of hyperparameter sets evaluated at the same time
                    n_jobs=1,                                         # number of jobs
                    iid=False,                                        # if not iid it optimizes on the cv score
                    return_train_score=False,                         
                    refit=False,                                      
                    optimizer_kwargs={'base_estimator': 'GP'},        # optmizer parameters: we use Gaussian Process (GP)
                    random_state=0)                                   # random state for replicability



In [13]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title="model", callbacks=None, eval_set=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    
    if callbacks is not None:
        optimizer.fit(X, y, callback=callbacks, eval_set=eval_set, eval_metric=['mlogloss','merror', 'auc'], early_stopping_rounds=60)
    else:
        optimizer.fit(X, y, eval_set=eval_set, eval_metric=['mlogloss','merror', 'auc'], early_stopping_rounds=60)
        
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           + u"\u00B1"+" %.3f") % (time() - start, 
                                   len(optimizer.cv_results_['params']),
                                   best_score,
                                   best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [14]:
# Running the optimizer
overdone_control = DeltaYStopper(delta=0.0001)                    # We stop if the gain of the optimization becomes too small
time_limit_control = DeadlineStopper(total_time=60*60*12)          # We impose a time limit (7 hours)

best_params = report_perf(opt, X_train, y_train,'XGBoost_classification', 
                          callbacks=[overdone_control, time_limit_control],
                         eval_set=eval_set)




Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3; 1/1] START colsample_bytree=0.8, learning_rate=0.05, max_depth=7, n_estimators=600, reg_alpha=0.42937191596882573, reg_lambda=29.416687880357603, subsample=0.7, verbose_eval=10
[12:11:18] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose_eval" } are not used.

[0]	validation_0-mlogloss:1.09906	validation_0-merror:0.64907	validation_0-auc:0.51199
[1]	validation_0-mlogloss:1.10278	validation_0-merror:0.67827	validation_0-auc:0.51411
[2]	validation_0-mlogloss:1.10713	validation_0-merror:0.67399	validation_0-auc:0.51253
[3]	validation_0-mlogloss:1.11086	validation_0-merror:0.67312	validation_0-auc:0.51324
[4]	validation_0-mlogloss:1.11596	validation_0-merror:0.67542	validation_0-auc:0.51333
[5]	validation_0-mlogloss:1.12044	validation_0-merror:0.66827	validation_0-auc:0.50980
[6]	validation_0-mlogloss:1.12634	validation_0-merror:0.67337	validation_0-auc:0.51011
[7]	validation_0-mlogloss:1.13018	validation_0-merror:0.

[22]	validation_0-mlogloss:1.14205	validation_0-merror:0.63392	validation_0-auc:0.48554
[23]	validation_0-mlogloss:1.14285	validation_0-merror:0.62897	validation_0-auc:0.48554
[24]	validation_0-mlogloss:1.14656	validation_0-merror:0.63260	validation_0-auc:0.48470
[25]	validation_0-mlogloss:1.15031	validation_0-merror:0.63204	validation_0-auc:0.48551
[26]	validation_0-mlogloss:1.15399	validation_0-merror:0.63148	validation_0-auc:0.48536
[27]	validation_0-mlogloss:1.15809	validation_0-merror:0.63189	validation_0-auc:0.48509
[28]	validation_0-mlogloss:1.16283	validation_0-merror:0.63234	validation_0-auc:0.48559
[29]	validation_0-mlogloss:1.16625	validation_0-merror:0.63500	validation_0-auc:0.48496
[30]	validation_0-mlogloss:1.17129	validation_0-merror:0.63610	validation_0-auc:0.48448
[31]	validation_0-mlogloss:1.17430	validation_0-merror:0.63566	validation_0-auc:0.48457
[32]	validation_0-mlogloss:1.17907	validation_0-merror:0.63334	validation_0-auc:0.48433
[33]	validation_0-mlogloss:1.183

[44]	validation_0-mlogloss:1.13584	validation_0-merror:0.58862	validation_0-auc:0.51600
[45]	validation_0-mlogloss:1.13741	validation_0-merror:0.58824	validation_0-auc:0.51674
[46]	validation_0-mlogloss:1.13857	validation_0-merror:0.58734	validation_0-auc:0.51693
[47]	validation_0-mlogloss:1.14050	validation_0-merror:0.58726	validation_0-auc:0.51660
[48]	validation_0-mlogloss:1.14550	validation_0-merror:0.58886	validation_0-auc:0.51647
[49]	validation_0-mlogloss:1.14918	validation_0-merror:0.58712	validation_0-auc:0.51656
[50]	validation_0-mlogloss:1.15554	validation_0-merror:0.58752	validation_0-auc:0.51602
[51]	validation_0-mlogloss:1.15839	validation_0-merror:0.58839	validation_0-auc:0.51588
[52]	validation_0-mlogloss:1.16424	validation_0-merror:0.58996	validation_0-auc:0.51592
[53]	validation_0-mlogloss:1.16754	validation_0-merror:0.58850	validation_0-auc:0.51568
[54]	validation_0-mlogloss:1.17027	validation_0-merror:0.58757	validation_0-auc:0.51565
[55]	validation_0-mlogloss:1.175

[1]	validation_0-mlogloss:1.10351	validation_0-merror:0.66153	validation_0-auc:0.49446
[2]	validation_0-mlogloss:1.10816	validation_0-merror:0.66174	validation_0-auc:0.48542
[3]	validation_0-mlogloss:1.10796	validation_0-merror:0.63668	validation_0-auc:0.48346
[4]	validation_0-mlogloss:1.11186	validation_0-merror:0.65546	validation_0-auc:0.48475
[5]	validation_0-mlogloss:1.11018	validation_0-merror:0.64734	validation_0-auc:0.48652
[6]	validation_0-mlogloss:1.10962	validation_0-merror:0.63703	validation_0-auc:0.48563
[7]	validation_0-mlogloss:1.11392	validation_0-merror:0.63912	validation_0-auc:0.48485
[8]	validation_0-mlogloss:1.11615	validation_0-merror:0.63214	validation_0-auc:0.48730
[9]	validation_0-mlogloss:1.12292	validation_0-merror:0.63166	validation_0-auc:0.48704
[10]	validation_0-mlogloss:1.12767	validation_0-merror:0.63832	validation_0-auc:0.48842
[11]	validation_0-mlogloss:1.13598	validation_0-merror:0.64001	validation_0-auc:0.48796
[12]	validation_0-mlogloss:1.13764	valida

[28]	validation_0-mlogloss:1.12547	validation_0-merror:0.59537	validation_0-auc:0.51732
[29]	validation_0-mlogloss:1.12954	validation_0-merror:0.59552	validation_0-auc:0.51768
[30]	validation_0-mlogloss:1.13473	validation_0-merror:0.59119	validation_0-auc:0.51826
[31]	validation_0-mlogloss:1.13932	validation_0-merror:0.59270	validation_0-auc:0.51852
[32]	validation_0-mlogloss:1.14270	validation_0-merror:0.59054	validation_0-auc:0.51797
[33]	validation_0-mlogloss:1.14985	validation_0-merror:0.58999	validation_0-auc:0.51808
[34]	validation_0-mlogloss:1.15491	validation_0-merror:0.59032	validation_0-auc:0.51861
[35]	validation_0-mlogloss:1.16000	validation_0-merror:0.58732	validation_0-auc:0.51909
[36]	validation_0-mlogloss:1.16605	validation_0-merror:0.59038	validation_0-auc:0.51985
[37]	validation_0-mlogloss:1.17069	validation_0-merror:0.58880	validation_0-auc:0.51882
[38]	validation_0-mlogloss:1.17573	validation_0-merror:0.59215	validation_0-auc:0.51744
[39]	validation_0-mlogloss:1.181

...

[21]	validation_0-mlogloss:1.11541	validation_0-merror:0.64877	validation_0-auc:0.49839
[22]	validation_0-mlogloss:1.11590	validation_0-merror:0.64668	validation_0-auc:0.49847
[23]	validation_0-mlogloss:1.11682	validation_0-merror:0.64590	validation_0-auc:0.49860
[24]	validation_0-mlogloss:1.11792	validation_0-merror:0.64771	validation_0-auc:0.49846
[25]	validation_0-mlogloss:1.11925	validation_0-merror:0.64919	validation_0-auc:0.49906
[26]	validation_0-mlogloss:1.11936	validation_0-merror:0.64552	validation_0-auc:0.49922
[27]	validation_0-mlogloss:1.12042	validation_0-merror:0.64597	validation_0-auc:0.49932
[28]	validation_0-mlogloss:1.12153	validation_0-merror:0.64734	validation_0-auc:0.49938
[29]	validation_0-mlogloss:1.12281	validation_0-merror:0.64704	validation_0-auc:0.49943
[30]	validation_0-mlogloss:1.12409	validation_0-merror:0.64642	validation_0-auc:0.49932
[31]	validation_0-mlogloss:1.12549	validation_0-merror:0.64721	validation_0-auc:0.49954
[32]	validation_0-mlogloss:1.126

[2]	validation_0-mlogloss:1.10048	validation_0-merror:0.70750	validation_0-auc:0.48554
[3]	validation_0-mlogloss:1.10152	validation_0-merror:0.71462	validation_0-auc:0.48523
[4]	validation_0-mlogloss:1.10256	validation_0-merror:0.73489	validation_0-auc:0.48429
[5]	validation_0-mlogloss:1.10364	validation_0-merror:0.74081	validation_0-auc:0.48417
[6]	validation_0-mlogloss:1.10476	validation_0-merror:0.73149	validation_0-auc:0.48452
[7]	validation_0-mlogloss:1.10597	validation_0-merror:0.74004	validation_0-auc:0.48354
[8]	validation_0-mlogloss:1.10679	validation_0-merror:0.74105	validation_0-auc:0.48455
[9]	validation_0-mlogloss:1.10685	validation_0-merror:0.73025	validation_0-auc:0.48439
[10]	validation_0-mlogloss:1.10765	validation_0-merror:0.73945	validation_0-auc:0.48586
[11]	validation_0-mlogloss:1.10827	validation_0-merror:0.72928	validation_0-auc:0.48737
[12]	validation_0-mlogloss:1.10881	validation_0-merror:0.72450	validation_0-auc:0.49028
[13]	validation_0-mlogloss:1.11012	valid

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[15]	validation_0-mlogloss:1.08695	validation_0-merror:0.59690	validation_0-auc:0.52732
[16]	validation_0-mlogloss:1.08768	validation_0-merror:0.59694	validation_0-auc:0.52715
[17]	validation_0-mlogloss:1.09116	validation_0-merror:0.59688	validation_0-auc:0.52741
[18]	validation_0-mlogloss:1.09343	validation_0-merror:0.59688	validation_0-auc:0.52709
[19]	validation_0-mlogloss:1.09884	validation_0-merror:0.59697	validation_0-auc:0.52721
[20]	validation_0-mlogloss:1.09619	validation_0-merror:0.59645	validation_0-auc:0.52571
[21]	validation_0-mlogloss:1.09758	validation_0-merror:0.59678	validation_0-auc:0.52450
[22]	validation_0-mlogloss:1.10054	validation_0-merror:0.59629	validation_0-auc:0.52437
[23]	validation_0-mlogloss:1.10211	validation_0-merror:0.59629	validation_0-auc:0.52289
[24]	validation_0-mlogloss:1.10381	validation_0-merror:0.59625	validation_0-auc:0.52196
[25]	validation_0-mlogloss:1.10632	validation_0-merror:0.59647	validation_0-auc:0.52219
[26]	validation_0-mlogloss:1.109

[26]	validation_0-mlogloss:1.11936	validation_0-merror:0.64479	validation_0-auc:0.49945
[27]	validation_0-mlogloss:1.12047	validation_0-merror:0.64480	validation_0-auc:0.50002
[28]	validation_0-mlogloss:1.12156	validation_0-merror:0.64541	validation_0-auc:0.49987
[29]	validation_0-mlogloss:1.12248	validation_0-merror:0.64695	validation_0-auc:0.49987
[30]	validation_0-mlogloss:1.12375	validation_0-merror:0.64566	validation_0-auc:0.49964
[31]	validation_0-mlogloss:1.12515	validation_0-merror:0.64699	validation_0-auc:0.49977
[32]	validation_0-mlogloss:1.12653	validation_0-merror:0.64677	validation_0-auc:0.50005
[33]	validation_0-mlogloss:1.12781	validation_0-merror:0.64592	validation_0-auc:0.50068
[34]	validation_0-mlogloss:1.12937	validation_0-merror:0.64689	validation_0-auc:0.50062
[35]	validation_0-mlogloss:1.13048	validation_0-merror:0.64747	validation_0-auc:0.50052
[36]	validation_0-mlogloss:1.13097	validation_0-merror:0.64754	validation_0-auc:0.50084
[37]	validation_0-mlogloss:1.132

In [15]:
print(best_params)

OrderedDict([('colsample_bytree', 0.9), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 389), ('reg_alpha', 0.5), ('reg_lambda', 1e-09), ('subsample', 0.8), ('verbose_eval', 10)])


In [8]:
df = read_parquet('/home/dcai/data/features/EURCHF/2/2021', '077.parquet')

df.info()
print(df['MeanPrice_x'].mean())
print(df['MeanPrice_y'].mean())

print(df['MeanPrice_x']-df['MeanPrice_y'])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12874 entries, 2021-03-18 08:00:00.017000 to 2021-03-18 20:59:56.241000
Data columns (total 77 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MeanPrice_x           12874 non-null  float64
 1   close_MIN_1D_5        12874 non-null  float64
 2   close_MIN_1D_20       12874 non-null  float64
 3   close_MIN_1D_60       12874 non-null  float64
 4   close_MAX_1D_5        12874 non-null  float64
 5   close_MAX_1D_20       12874 non-null  float64
 6   close_MAX_1D_60       12874 non-null  float64
 7   close_AVG_1D_5        12874 non-null  float64
 8   close_AVG_1D_20       12874 non-null  float64
 9   close_AVG_1D_60       12874 non-null  float64
 10  close_STDEV_1D_5      12874 non-null  float64
 11  close_STDEV_1D_20     12874 non-null  float64
 12  close_STDEV_1D_60     12874 non-null  float64
 13  close_MEDIAN_1D_5     12874 non-null  float64
 14  close_MEDIAN_1D_20   